In [19]:
import numpy as np
from typing import Tuple, Callable, Protocol
from enum import Enum
import math
import sympy

In [3]:
class Pivoting(Enum):
  NO_PIVOTING = 0,
  PARTIAL_PIVOTING = 1,
  FULL_PIVOTING = 2

def do_partial_pivoting(A: np.ndarray, b:np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  # find hihest row value
  index_highest_values_per_column = np.abs(A).argmax(1)
  index_hihest_first_column = index_highest_values_per_column[0]

  # Swap rows
  A[[0, index_hihest_first_column]] = A[[index_hihest_first_column, 0]]
  b[[0, index_hihest_first_column]] = b[[index_hihest_first_column, 0]]

  return (A, b)

def do_partial_pivoting_lu(upper_matrix : np.ndarray, permutation_matrix: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  index_highest_values_per_column = np.abs(upper_matrix).argmax(1)
  index_hihest_first_column = index_highest_values_per_column[0]

  # return a permutation matrix
  # permutation_matrix = np.identity(len(upper_matrix))
  permutation_matrix[[0, index_hihest_first_column]] = permutation_matrix[[index_hihest_first_column, 0]]

  # Swap rows
  upper_matrix[[0, index_hihest_first_column]] = upper_matrix[[index_hihest_first_column, 0]]
  
  return (upper_matrix, permutation_matrix)

def do_full_pivoting(A : np.ndarray, b:np.ndarray) -> Tuple[np.ndarray, np.ndarray, int]:
  # Find the highest value
  highest_value = max(A.min(), A.max(), key=abs)
  max_value = np.where(A == highest_value)

  # Use the first found occurence if there is more than one
  index_row_highest_value = max_value[0][0]
  index_column_highest_value = max_value[1][0]

  # Swap rows
  A[[0, index_row_highest_value]] = A[[index_row_highest_value, 0]]
  b[[0, index_row_highest_value]] = b[[index_row_highest_value, 0]]

  # Swap columns
  A[:, [0, index_column_highest_value]] = A[:, [index_column_highest_value, 0]]

  return (A, b, index_column_highest_value)

# Gaussian Elimination method

This method for solving linear systems works by forming a superior triangle matrix 

In [4]:
def gauss_elimination(A: np.ndarray, b:np.ndarray, pivoting = Pivoting.NO_PIVOTING) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  exchanged_columns = []

  for index_col in range(len(A)):
    match pivoting:
      case Pivoting.NO_PIVOTING:
        pass
      case Pivoting.PARTIAL_PIVOTING:
        A[index_col:, index_col:] , b[index_col:] = do_partial_pivoting(A[index_col:, index_col:] , b[index_col:])
      case Pivoting.FULL_PIVOTING:
        A[index_col:, index_col:], b[index_col:], local_column_excanged = do_full_pivoting(A[index_col:, index_col:] , b[index_col:])
        exchanged_columns.append(local_column_excanged)
      
    pivot = A[index_col][index_col]
    for index_row_below in range(index_col + 1, len(A)):
      ratio = A[index_row_below][index_col] / pivot
      
      for idx in range(index_col, len(A)):
        A[index_row_below][idx] -= A[index_col][idx] * ratio

      b[index_row_below] -= b[index_col] * ratio
      

  result = np.linalg.solve(A, b)
  if pivoting == Pivoting.FULL_PIVOTING:
    for index, excahnged in enumerate(exchanged_columns):
      result[[index, index + excahnged]] = result[[index + excahnged, index]]

  return result

# LU Factoring Method

In [5]:
def is_possible_to_apply_LU_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    if A.shape == (0,0):
      return True
    return np.linalg.det(A) != 0 and is_possible_to_apply_LU_method(A[1:, 1:])
  return False

# https://jonshiach.github.io/ODEs-book/pages/5.3_LUP_decomposition.html
def lu_factoring(A: np.ndarray, b:np.ndarray, pivoting = Pivoting.NO_PIVOTING) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  assert is_possible_to_apply_LU_method(A) == True

  permutation_matrix = np.identity(len(A), np.float64)

  match pivoting:
    case Pivoting.NO_PIVOTING:
      pass
    case Pivoting.PARTIAL_PIVOTING:
      # Calculate pivoting matrix
        copy_of_A = A.copy()
        for index_col in range(len(A)):
          copy_of_A[index_col:, index_col:], permutation_matrix[index_col:, :]  = do_partial_pivoting_lu(copy_of_A[index_col:, index_col:], permutation_matrix[index_col:, :])
    case Pivoting.FULL_PIVOTING:
      raise NotImplemented("Not implemented for LU")
    
  upper_matrix = np.dot(permutation_matrix, A.copy())
  lower_matrix = np.identity(len(A), np.float64)
  
  for index_col in range(len(upper_matrix)):
    pivot = upper_matrix[index_col, index_col] 
    
    for index_row_below in range(index_col + 1, len(upper_matrix)):
      ratio = upper_matrix[index_row_below, index_col]/pivot
      lower_matrix[index_row_below, index_col] = ratio

      for idx in range(index_col, len(A)):
        upper_matrix[index_row_below, idx] -= upper_matrix[index_col, idx] * ratio

  partial_result = np.linalg.solve(lower_matrix, np.dot(permutation_matrix, b))
  result = np.linalg.solve(upper_matrix, partial_result)
  return result

# Cholesky method

In [6]:
def is_possible_to_apply_Cholesky_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    if A.shape == (0,0):
      return True
    return np.linalg.det(A) > 0.0 and np.allclose(A, A.T) and is_possible_to_apply_Cholesky_method(A[1:, 1:])
  return False

def cholesky_method(A:np.ndarray, b:np.ndarray) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  assert is_possible_to_apply_Cholesky_method(A) == True

  G = np.zeros(A.shape)

  for i in range(len(A)):
    for j in range(i + 1):
      sum = 0
      for k in range(j):
        sum += G[i][k] * G[j][k]
      
      if i == j:
        G[i][j] = math.sqrt(A[i][i] - sum)
      else:
        G[i][j] = (A[i][j] - sum) / G[j][j]

  G_transpose = G.T
  y = np.linalg.solve(G, b)
  x = np.linalg.solve(G_transpose, y)
  return x

# Gauss Jacobi

In [7]:
def is_possible_to_apply_Jacobi_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    spectral_radius_less_than_one = all(map(lambda x: abs(x) < 1.0, np.linalg.eigvals(A)))
    
    rows_criteria = all(map(lambda x: A[x, :].sum() - A[x, x] < A[x, x], range(len(A))))
    columns_criteria = all(map(lambda x: A[:, x].sum() - A[x, x] < A[x, x], range(len(A))))

    diagonally_dominant = rows_criteria or columns_criteria

    return spectral_radius_less_than_one or diagonally_dominant
  return False

def jacobi_method(A:np.ndarray, b:np.ndarray, x:np.ndarray, iterations: int = 10) -> np.ndarray:
  assert is_possible_to_apply_Jacobi_method(A) == True

  for _ in range(iterations):
    # Diagonal matrix
    D = np.diag(np.diag(A))
    
    # construct a diagonal matrix where each position is 1/(diagonal element of A)
    D_inv = np.linalg.inv(D)

    x = np.dot(D_inv, b - np.dot(A - D, x))

  return x

# Gaus Seidel

In [8]:
def is_possible_to_apply_Seidel_method(A:np.ndarray) -> bool:
  return is_possible_to_apply_Jacobi_method(A) # and meet_sassenfeld_criterion(A)

def seidel_method(A:np.ndarray, b: np.ndarray, x: np.ndarray, iterations : int = 10) -> np.ndarray:
  assert is_possible_to_apply_Seidel_method(A) == True

  for _ in range(iterations):
    for i in range(len(A)):
      x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

  return x

# Newton's Method

In [20]:
def is_possible_to_apply_Newton_method(A:np.ndarray) -> bool:
  pass

class Callback(Protocol) :
  def __call__(self, *args:np.float64) -> np.float64: ...

def callback(*args:np.float64) -> np.float64:
  return args[0] * 3.0

"""
F is an one dimensional array where each entry is a function that takes n variables (Callback)
n is the number of variables on each function

Ex.:
F = [
  lambda x, y: x**2 + sin(y) + 3,
  lambda x, y: sqrt(x) + y - 9,
]
n = 2   # x and y
"""
def generate_jacobi_function(F: np.ndarray[Callback], n:int) -> Callable[[np.ndarray], np.ndarray]:
  assert F.ndim == 1

  # Jacobi matrix is one that each entry is defined as a function
  # that will return a value for a given input
  jacobi_matrix: np.ndarray[Callable[[np.float64], np.float64]]

  for row in range(len(F)):
    for column in range(n):
      # Calculate the partial derivative in relation to the 
      # funciton in the row_th row and the column_th variable
      # jacobi_matrix[row, column] = np.gradient(F[row, column])

  def temp(x: np.ndarray) -> np.ndarray:
    map(jacobi_matrix, x)
    return 
  
  return temp

def newton_method(F: Callable[[np.ndarray], np.ndarray], J: Callable[[np.ndarray], np.ndarray], x:np.ndarray, iterations: int = 10) -> np.ndarray:
  
  pass


# Gauss with pivoting

In [10]:
A = np.array([[1,-7], [5,2]], dtype=np.float64)
b = np.array([-11,-18], dtype=np.float64)
# Solution x=-4, y=1
gauss_elimination(A,b, Pivoting.PARTIAL_PIVOTING)

array([-4.,  1.])

In [11]:
A = np.array([
  [3,2,4],
  [1,1,2],
  [4,3,-2],], 
dtype=np.float64)
b = np.array([1,2,3], np.float64)

# {-3, 5, 0}
lu_factoring(A, b, Pivoting.PARTIAL_PIVOTING)

array([-3.,  5.,  0.])

In [12]:
# 1x + 2y - 2z = -15
# 2x + 1y - 5z = -21
# 1x - 4y + 1z = +18 
# Solution {-1, -4, 3}

A = np.array([
  [1, 2, -2],
  [2, 1, -5],
  [1, -4, 1]
], dtype=np.float64)

b = np.array([-15, -21, 18], dtype=np.float64)

gauss_elimination(A, b, Pivoting.FULL_PIVOTING)

array([-1., -4.,  3.])

In [13]:
A = np.array([
  [1.,    2.,     1., -1.],
  [3./2., 1.,     2., 2.],
  [4.,    4.,     3., 4.],
  [2./5., 0.,  1./5., 1.]
], dtype=np.float64)

b = np.array([5., 8., 22., 3.], dtype=np.float64)

# x = 16
# y = -6
# z = -2
# t = -3

gauss_elimination(A, b, Pivoting.FULL_PIVOTING)

array([16., -6., -2., -3.])

In [14]:
A = np.array([
  [1.,  1.,  0.],
  [1.,  2., -1.],
  [0., -1.,  3.],
], dtype=np.float64)

b = np.array([2., 1., 5.], dtype=np.float64)

# x = 1
# y = 1
# z = 2

cholesky_method(A, b)

array([1., 1., 2.])

In [15]:
A = np.array([
  [4.,  2.,  -4.],
  [2.,  10., 4.],
  [-4., 4.,  9.],
], dtype=np.float64)

b = np.array([0., 6., 5.], dtype=np.float64)

# x = 1
# y = 0
# z = 1

cholesky_method(A, b)

array([1., 0., 1.])

In [16]:
A = np.array([
  [4.,  2.,  -4.],
  [2.,  10., 4.],
  [-4., 4.,  9.],
], dtype=np.float64)

b = np.array([0., 6., 5.], dtype=np.float64)

jacobi_method(A, b, np.array([0., 0., 0.]), 20)

array([0.63911074, 0.18195205, 0.74674592])

In [17]:
A = np.array([
  [4.,  2.,  -4.],
  [2.,  10., 4.],
  [-4., 4.,  9.],
], dtype=np.float64)

b = np.array([0., 6., 5.], dtype=np.float64)

seidel_method(A, b, np.array([0., 0., 0.]), 20)

array([0.82057774, 0.08823726, 0.88104021])